The below is the main code for generating Gradient Boost for classification tree.
The basic idea is from 
https://www.youtube.com/watch?v=jxuNLH5dXCs&t=2s


# Data Input

In [1]:
import numpy as np
import pandas as pd 
data= pd.read_excel('C:/Users/shufe/Dropbox/ds new/pwchkt.xlsx')
data.isnull().any(axis=1) 
data=data.dropna(how='any')
data=data[['age','creditamount','guarantors','housing','foreigner','creditrating']]
data.loc[(data.creditrating == 2),'creditrating']=0
data_features_original = list(data.columns) 
y='creditrating'
data_features_original.remove(y)
# we also need to create a dictionary indicating whether the feature is discrete or not
discrete={'age':'no','creditamount':'no','guarantors':'yes','housing':'yes','foreigner':'yes','creditrating':'yes'}
data_features=data_features_original.copy()

# Define Main Function

In [2]:
def GradientBoost (data , max_tree_number,learn_rate ):
    n=len(data)
    current_data = data.copy()
    current_tree_series = []
    current_tree_series.append( math.log( sum ( data [ y ] == 1 )/ sum ( data[ y ] == 0 ) ) )
    current_odd = [math.log( sum ( data [ y ] == 1 )/ sum ( data[ y ] == 0 ) ) for i in range(0,n)]
    current_data['pre_predict']=[  math.exp(f)/(math.exp(f)+1)     for f in current_odd]
    while (len(current_tree_series)<=max_tree_number):
       
        data_features = data_features_original.copy()
       # thorough this step you get a new tree. the leaves of the tree stores odd value
        new_tree = createTree ( current_data, 1, data_features) # a tree always start from first layer
      # add the newly generated tree into the current tree respository
        current_tree_series.append ( new_tree )
       # produce new odd value
        new_odd = [ output ( current_data.loc[i],new_tree ) for  i in range(0,n) ]
       # add to the current odd value with a learning rate
        current_odd = list(map(lambda x,y:x+y, current_odd , [j * learn_rate for j in new_odd] ))
       # using the current odd value, you can produce a up-to-now prediction for each individual
        current_data['pre_predict']=[  math.exp(f)/(math.exp(f)+1)   for f in current_odd]
       # print(sum(list(map(lambda a,b: abs(a-b), current_data[y],current_data['pre_predict']))))
    return current_tree_series

# Base Classifier: Tree Generation

## Main function

In [3]:
def createTree ( Dataset, layer,features):
    # Dataset contains the real y and the previous predict (p, not odd value)
    # we first need to check whether the layer has already reached the max
    
    if ( layer == maxlayer ):
        # you have already reached the deepest. 
        # you only have to return the gama, as the final result of this leaf.
        Tree = loss_cal ( Dataset[y] , Dataset['pre_predict'])[0]
    else:
        best_feature,best_threshold = BestFeature ( Dataset, features)
        Tree = {}  # means that you want to generate a new son tree
        # the new son tree should contain the following features:
        Tree['spInd'] = best_feature # the feature you use to split the data
        featuresNext = features
        featuresNext.remove ( best_feature )  # this feature would no longer be used in the next trees
        Tree['spVal'] = best_threshold # the threshold value 
        data_left, data_right = SplitDataSet(
            Dataset,best_feature,best_threshold)[1:3]
        Tree['left'] = createTree ( data_left, layer+1,featuresNext)
        Tree['right'] = createTree ( data_right, layer+1,featuresNext)
    return Tree

## Select Best Feature

In [4]:
n_seq=8
def BestFeature ( Dataset, unvisited_feature ):
    # want to return the best feature and the resulting data split
    # but before this , you can already calculate the the entropy of the current data
    threshold=[]
    loss=[]
    
    for feature in unvisited_feature:
        if discrete[feature]=='yes': # this is discrete 
            # feature is name
            # next, for each level in the feature_level, get
            loss.append ( SplitDataSet ( Dataset,feature, 0.5)[0])
            threshold.append(0.5)
        else: # the feature is continuous 
            # we first generate a threshold sequence 
            thres_seq = [   min(Dataset[feature])+i*(max(Dataset[feature])-min(Dataset[feature]))/(n_seq-1) 
                      for i in range(1,n_seq-1)   ]
            loss_seq= [ SplitDataSet(Dataset,feature, x) [0] for x in thres_seq]
            loss.append ( min ( loss_seq ))
            threshold.append ( thres_seq [ loss_seq.index( min ( loss_seq) ) ])
        
    # we complete the loop with loss series
    # we now return the best feature and the corresponding threshold 
    best_feature = unvisited_feature [ loss.index(min(loss)) ]
    best_threshold = threshold [ loss.index(min(loss)) ]
    # we also return the splitted dat
    return best_feature,best_threshold
    # best_feature is a string. best_threshold is a value. 
  


## Split data by feature

In [5]:
def SplitDataSet ( Dataset,feature, value ):
# we only consider the binary classification case
# given the dataset, feature and value, this function returns splitted data and the resulting entropy!
    
    Data_left = Dataset[ Dataset[feature]<value ]
    Data_right = Dataset[ Dataset[feature]>=value ]
    
# we want to calculate the loss function the data_left, data_right
    
    if (len(Data_left)*len(Data_right)>0): 
        Total_loss = loss_cal ( Data_left[y] , Data_left['pre_predict']
                          )[1] + loss_cal ( Data_right[y] , Data_right['pre_predict'] )[1]
    else: # if any of the sub data is null, then the current (feature, value) split is not valid. set a inf value to the loss function
        Total_loss=float("inf")
    
    return Total_loss,Data_left, Data_right

## Loss Function due to classfication

In [6]:
import math
def loss_cal ( y_data , pre_predict ):
    # this function gives you the output of loss function as well as the gamma, the odd ratio at the leaf.
    # y_data is already the residual!
    residual=y_data-pre_predict
    gama = sum ( residual ) / ( sum ( [  p*(1-p)  for p in pre_predict ] )  )
    # the expression for gama is actually the solution to the maximization of the following loss function.
    loss = sum ( list ( map( lambda y,p :-y*math.log(p)-(1-y)*math.log(1-p),y_data, pre_predict) ))
    return gama , loss
    

## Output the Odd ratio of a sample given a tree

In [7]:
def output(sample, tree):
    if (sample[tree['spInd']]<tree['spVal']):
        odd = output(sample,tree['left']) if type(tree['left']).__name__=='dict' else tree['left']
    else:
        odd = output(sample,tree['right']) if type(tree['right']).__name__=='dict' else  tree['right']
    return (odd)

# Test

In [ ]:
maxlayer=3
GradientBoost (data , 300,0.3 )